In [1]:
import MetaTrader5 as mt5
import datetime
from datetime import datetime, timedelta, date
import isoweek
import time
import pandas as pd
import numpy as np
import talib as ta
import os

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

import schedule
import pytz
from IPython.display import clear_output

In [2]:
account = 54921576
password = "hmatfhh0"

#Connect MetaTrader
def connect(account):
    mt5.initialize()
    authorized = mt5.login(account, password="{}".format(password), server="MetaQuotes-Demo")

    if authorized:
        print("Connected, time : " + str(datetime.now().strftime('%d/%m/%Y %H:%M:%S')))
    else:
        print("Failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
        
connect(account)

Connected, time : 19/03/2022 17:46:01


In [3]:
%matplotlib tk
def open_position(order_type, size=0.01, spread=0 ,pair='XAUUSD'):
    symbol_info = mt5.symbol_info(pair)
    point = symbol_info.point
    if symbol_info is None:
        return
            
    if not symbol_info.visible:
        print(pair, "is not visible, trying to switch on")
        if not mt5.symbol_select(pair, True):
            print("symbol_select({}}) failed, exit",pair)
            return
    print("pair found :" + pair)
    print(symbol_info.point)
    
    tp_distance = 9 + spread / 10
    sl_distance = 20 + spread / 10

    if(order_type == "BUY"):
        order = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(pair).ask
        if(sl_distance):
            sl = price - sl_distance
        if(tp_distance):
            tp = price + tp_distance
    if(order_type == "SELL"):
        order = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(pair).bid
        if(sl_distance):
            sl = price + sl_distance
        if(tp_distance):
            tp = price - tp_distance
            
    if order_type != 'BUY' or order_type != 'SELL': 
        return 'order_type_Error: try "BUY" or "SELL"'

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": pair,
        "volume": float(size),
        "type": order,
        "price": price,
      "sl": sl,
      "tp": tp,
        "magic": 234000,
        "comment": order_type,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
            }

    result = mt5.order_send(request)

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Failed to send order :(")
        print ('Reason : ' + str(result.comment))
    else:
        print ("Order successfully placed!")

def positions_get(symbol=None):
    if(symbol is None):
        res = mt5.positions_get()
    else:
        res = mt5.positions_get(symbol=symbol)

    if(res is not None and res != ()):
        df = pd.DataFrame(list(res),columns=res[0]._asdict().keys())
        df['time'] = pd.to_datetime(df['time'], unit='s')
        #df = pd.concat([df['time'], df['price_open'], df['sl'], df['tp'], df['price_current'], df['identifer']], axis=1)
        return df
    
    
def close_position(deal_id):
    open_positions = positions_get()
    open_positions = open_positions[open_positions['ticket'] == int(deal_id)]
    order_type  = open_positions["type"][0]
    symbol = open_positions['symbol'][0]
    volume = open_positions['volume'][0]

    if(order_type == mt5.ORDER_TYPE_BUY):
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
        
    close_request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(volume),
        "type": order_type,
        "position": int(deal_id),
        "price": price,
        "magic": 234000,
        "comment": "Close trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(close_request)
    
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Failed to close order :(")
    else:
        print ("Order successfully closed!")
    
def live_trading():
    print('-----live_trading-----')
    #schedule.every().hour.at(":00").do(run_trader)
    #schedule.every().hour.at(":30").do(run_trader)
    schedule.every(3).minutes.at(":00").do(run_trader)

    while True:
        schedule.run_pending()
        time.sleep(1)
        
        
def get_data(pair='XAUUSD', time_frame=mt5.TIMEFRAME_H1):

    #Depends on the timezone of the broker, timezone may vary
    #Data from last 2 years
    from_date = datetime.now().astimezone(pytz.timezone('Asia/Singapore')
                                         ) - timedelta(weeks=isoweek.Week.last_week_of_year(datetime.now().year).week) * 2
    
    #To today
    to_date = datetime.now().astimezone(pytz.timezone('Asia/Singapore')) #plus one hour
    try:
        to_date = datetime(to_date.year, to_date.month, to_date.day, hour=to_date.hour + 1, minute=to_date.minute)
    except ValueError:
        to_date = datetime(to_date.year, to_date.month, to_date.day, hour=to_date.hour, minute=to_date.minute)
    
    
    rates = mt5.copy_rates_range(pair, time_frame, from_date, to_date)
    
    rates_frame = pd.DataFrame(rates)
    df_hour = pd.DataFrame(rates_frame)
    df_hour['time'] = pd.to_datetime(df_hour['time'], unit='s')
    
    df_close = pd.concat([df_hour.close, df_hour.low, df_hour.high, df_hour.time, df_hour.spread], axis=1)        
    df_close['MA_hour'] = round(ta.SMA(df_close['close'], 50), 2)
    df_close['MA_day'] = round(ta.SMA(df_close['close'], 50 * 24), 2)
    df_close['MA_week'] = round(ta.SMA(df_close['close'], 50 * 24 * 7), 2)
    df_close['average'] = df_close.close.rolling(24).mean().shift()
    
    #Drop 95% of the rows
    df_close = df_close.drop(np.arange(int(len(df_close) * 98 / 100)))
    
    df_close['close-MA_distance'] = abs(df_close.close - df_close.MA_hour)
    df_close['average-MA_distance'] = round(abs(df_close.average - df_close.MA_hour), 2)
    df_close['close_to_MA'] = np.where((df_close['close-MA_distance'] < df_close['average-MA_distance']),
                                  True, False)
    
    df_close_copy = df_close['close'].copy()[::-1]
    df_close_time_copy = df_close['time'].copy()[::-1]
    
    next_min = pd.Series(df_close_copy.rolling(24).min().shift(), name='next_min')
    next_max = pd.Series(df_close_copy.rolling(24).max().shift(), name='next_max')
    df_close_copy = pd.concat([df_close_copy, df_close_time_copy, next_min, next_max], axis=1)
    
    df_close_copy_drop_next_min = df_close_copy.drop_duplicates(subset=['next_min'], keep='last')
    df_close_copy_drop_next_min.columns = ['close', 'next_min_time', 'next_min_nodup', 'next_max']
    
    df_close_copy_drop_next_max = df_close_copy.drop_duplicates(subset=['next_max'], keep='last')
    df_close_copy_drop_next_max.columns = ['close', 'next_max_time', 'next_min', 'next_max_nodup']

    df_close_copy_drop_next_min = df_close_copy_drop_next_min[::-1]
    df_close_copy_drop_next_max = df_close_copy_drop_next_max[::-1]
    
    df_close = pd.concat([df_close, next_min, next_max,
                          df_close_copy_drop_next_min.next_min_nodup,
                          df_close_copy_drop_next_max.next_max_nodup,
                          df_close_copy_drop_next_min.next_min_time,
                         df_close_copy_drop_next_max.next_max_time], axis=1)
    df_close['next_min_time'] = df_close['next_min_time'].fillna(method='bfill')
    df_close['next_max_time'] = df_close['next_max_time'].fillna(method='bfill')
    
    df_close['Result'] = np.where((((df_close['close'] - df_close['next_min'] < 1.5) & #next min price not hitting SL
                                  (df_close['next_max'] - df_close['close'] > 9)) | #next max price exceed TP
                                  ((df_close['next_max'] - df_close['close'] > 9) &
                                  (df_close['next_max_time'] > df_close['next_min_time'])) |
                                   
                                ((df_close['close'] - df_close['next_max'] > -1.5) &
                                (df_close['next_min'] - df_close['close'] < -9) |
                                (df_close['next_min'] - df_close['close'] < -9) &
                                  (df_close['next_max_time'] < df_close['next_min_time']))), 1, 0)
    
    
    df_close.loc[((df_close.close > df_close.MA_hour) &
                (df_close.close > df_close.MA_day) &
                (df_close.close > df_close.MA_week)), 'position_type'] = 'BUY'

    df_close.loc[((df_close.close < df_close.MA_hour) &
                (df_close.close < df_close.MA_day) &
                (df_close.close < df_close.MA_week)), 'position_type'] = 'SELL'
    
    
    cluster_date(df_close)
    
    df_close = df_close.reset_index().drop(['index', 'average', 'close-MA_distance', 'average-MA_distance',
                                           'next_min_nodup', 'next_max_nodup', 'next_min', 'next_max',
                                           'next_min_time', 'next_max_time'], axis=1)
    
    global all_pos
    all_pos = pd.concat([df_close['close'], df_close['time'], df_close['MA_hour']], axis=1)

    
    path = r'C:\Python\csv_files\trading_data.csv'
    isExist = os.path.exists(path)
    
    if isExist:
        
        csv_pos = pd.read_csv(r'C:\Python\csv_files\trading_data.csv')
        csv_pos['time'] = pd.to_datetime(csv_pos['time'])
        all_pos['time'] = pd.to_datetime(all_pos['time'])
        pos = pd.concat([all_pos, csv_pos], axis=0, ignore_index=True)
        pos = pos.drop_duplicates(subset=['time'], keep='last')
        pos = pos.sort_values('time')
        pos.to_csv(r'C:\Python\csv_files\trading_data.csv', index=False)
    else:
        all_pos.drop('MA_hour', axis=1).to_csv(r'C:\Python\csv_files\trading_data.csv', index=False)
    
    print(datetime.now())
    
    
    return df_close

def check_trades():
    df_close = get_data()
    predict = time_series_forecast(all_pos)
    display(df_close)
    if int(df_close['valid'].iloc[-1]) == 1 and df_close['position_type'].iloc[-1] != np.nan and predict is not None:
        print(f'Prediction : {predict}')
        if predict == df_close['position_type'].iloc[-1]:
            open_position(df_close['position_type'].iloc[-1], size=0.01, spread=df_close['spread'].iloc[-1] ,
                          pair='XAUUSD')
            print('position placed : ' + datetime.now().strftime('%d/%m/%Y %H:%M:%S'))       
        
    if not positions_get() is None:
        for pos in range(len(positions_get())):
            stop_loss = positions_get()['sl'].iloc[pos]
            price_current = positions_get()['price_current'].iloc[pos]
            sl_dist = abs(stop_loss - price_current)
            if sl_dist < 18.5:
                close_position(positions_get()['ticket'].iloc[pos])
                
    
def run_trader():
    clear_output(wait=True)
    connect(account)
    get_data()
    check_trades()
    
def cluster_date(df):
    df['valid'] = np.nan
    valid_arr = np.array([])
    valid = np.array([])
    if len(df) < 20:
        cluster_range = len(df)
    else:
        cluster_range = 20
    
    for num2 in range(1, cluster_range + 1):
            
        if (abs(df['time'].iloc[-1] - df['time'].iloc[-1 - num2]) < timedelta(days=1)
            ) and (df['Result'].iloc[-2 - num2] == 0):
            #-2, not including new position itself
            valid = np.append(valid, False)
        else:
            valid = np.append(valid, True)
        
        valid_arr = np.append(valid_arr, all(valid))
        
    with pd.option_context('mode.chained_assignment',None):
        df['valid'].iloc[-cluster_range:] = valid_arr
        
def time_series_forecast(test=None, plot=None):
    inputs = 12
    test_x = np.zeros([len(test), 12])
    zero = np.zeros([1])
    outputs = 6
    time_series = test['time']
    MA_hour = test['MA_hour']
    test = pd.concat([pd.DataFrame(test['close']), pd.DataFrame(zero, columns=['close'])], axis=0)
    test = test.squeeze()

    for num2 in range(len(test_x)):
        try:
            test_x[-num2-1] = test.iloc[-num2-1 - inputs:-num2-1]
        except ValueError:
            test_x[test_x == 0] = np.nan
            test_x = test_x[~np.isnan(test_x).any(axis=1)]
            break


    test_x = pd.DataFrame(test_x)
    test = pd.DataFrame(test)
    
    test = test.values.reshape(-1, 1)
    MA_hour = MA_hour.values.reshape(-1, 1)
    
    #Normalize data
    scaler = MinMaxScaler(feature_range = (0, 1))
    test_x = scaler.fit_transform(test_x)
    
    test = test[:-1, :]
    test = scaler.fit_transform(test)
    MA_hour = scaler.fit_transform(MA_hour)
    
    #Load model
    loaded_model = tf.keras.models.load_model(r'C:\Python\tensorflow_models\algotrading model')
    
    
    y_pred = loaded_model.predict(test_x, batch_size=1)
    zeros = np.zeros([outputs - 1, 1])
    zeros[zeros == 0] = np.NaN
    y_pred = np.insert(y_pred, 0, zeros, axis=0)

    time = time_series.iloc[:len(test)]
    time2 = time_series.iloc[:len(y_pred)]
    ma_time = time_series.iloc[:len(MA_hour)]
    current_position = [test[-1]]
    
    pred = y_pred[len(y_pred) - 6:]
    pred = np.mean(pred)
    pred_buy = pred > current_position
    pred_sell = pred < current_position
    
    
    if plot is not None:
        fig, ax = plt.subplots(2, 1, figsize=(14, 5))
        
        true_plot = ax[0].plot(time, test)
        predict_plot = ax[0].plot(time2, y_pred)
        ma = ax[0].plot(ma_time, MA_hour)
        
        true_plot_zoom = ax[1].plot(time, test)
        predict_plot_zoom = ax[1].plot(time2, y_pred)
        ma_zoom = ax[1].plot(ma_time, MA_hour)
        
        current_position = test[-1]
        current_time = time.iloc[-1]

        ax[1].set_xlim(time2.loc[round(len(time2) * 70 / 100, 0)], time2.iloc[-1])

        ax[0].scatter(current_time, current_position,c='r', label='current_position', marker='^')
        ax[0].legend(['true', 'predict', 'MA_hour', 'current_position'])
        ax[0].set(xticklabels=[])
        ax[0].set(yticklabels=[])
    
        ax[1].scatter(current_time, current_position,c='r', label='current_position', marker='^')
        ax[1].legend(['true', 'predict', 'MA_hour', 'current_position'])
        ax[1].set(xticklabels=[])
        ax[1].set(yticklabels=[])

        plt.rcParams['backend'] = 'TkAgg'
        plt.show(block=True)

    
    if pred_buy > pred_sell:
        return 'BUY'
    elif pred_buy < pred_sell:
        return 'SELL'
    else:
        return None

        

In [ ]:
#if __name__ == '__main__':
#    live_trading()

In [ ]:
#display(positions_get())

In [ ]:
#close_position()

In [ ]:
#open_position('SELL')